In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3249385629782303598
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


c:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Derma


In [2]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv


from hanspell import spell_checker

from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [3]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [4]:
data_org = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\Lbl2Vec\\data_derma\\[0503]data_for_Lvl2Vec.xlsx",index_col=0)
print(data_org.shape)
data_org.head()

(29363, 10)


,h,adr,reviews,stars,treatments,money,adr_else,dates,cleaned_reviews,stemmed
0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,두피 가려움증,4900,57,2022-03-15,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"['두피/Noun', '가려움증/Noun', '으로/Josa', '내원한/Verb'..."
1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10.0,대상포진,4200,57,2021-12-20,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,"['대상포진/Noun', '으로/Josa', '진료/Noun', '받았어요/Verb..."
2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10.0,관리 안 됨,4000,57,2021-08-11,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"['여드름/Noun', '고민/Noun', '이/Josa', '있어서/Adjecti..."
3,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10.0,문신제거,22000,57,2021-06-21,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"['문신/Noun', '제거/Noun', '문신/Noun', '한/Josa', '곳..."
4,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,피부 트러블,4000,57,2021-06-07,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"['원장/Noun', '님/Suffix', '이/Determiner', '설명/No..."


In [5]:
def correction(x):
    try:
        x = x.replace('대기 시간', '대기시간')
        x = x.replace('점심 시간', '점심시간')
        x = x.replace('진료 시간', '진료시간')
        x = x.replace('치료 시간', '치료시간')
        x = x.replace('관리 시간', '관리시간')
    except:
        pass
    return x

In [6]:
data = data_org.copy()
data = data[['h', 'cleaned_reviews']]

data['cleaned_reviews'] = data['cleaned_reviews'].apply(correction)
# data['work'] = data['h']+'_'+data['cleaned_reviews']
data.head()

,h,cleaned_reviews
0,고운나라피부과의원,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...
1,고운나라피부과의원,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...
3,고운나라피부과의원,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...
4,고운나라피부과의원,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...


In [7]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = False)
    # print(pos)
    try:
        for j in pos:
            if j.split('/')[1] == 'Noun':
                j = j.split('/')[0]
                morphs_list.append(j)
                        
            elif j.split('/')[1] =='Adjective':
                k = okt.morphs(j,  stem= True)
                k = k[0]
                morphs_list.append(k)
            elif j.split('/')[1] =='Verb':
                v = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(v)

            elif j.split('/')[1] =='Adverb':
                ad = okt.morphs(j,  stem= True)
                # v = v[0]
                morphs_list.append(ad)

        for i in morphs_list:
            if len(i) != 1:
                one_words.append(i)
        

        for i in one_words:
            if i not in stopwords_kor:
                result.append(i)
    except:
        result.append(text)
        pass

    return result

In [8]:
from hanspell import spell_checker

In [9]:
data['work'] = data['h']+'_'+data['cleaned_reviews']

In [11]:
test = []
hospital_list = []

for i in tqdm(data['work'].tolist()):
    try:
        hos = i.split('_')[0]
        i = i.split('_')[-1]
        # print(i)
        i = i.replace('네요','다')
        i = i.replace('해요','')
        pos = okt.pos(i, join = True)
        # print(pos)
        sen = str(pos).split("/Verb',")
        for i in sen:
            i = clean_text(i)
            i = i.replace(' ','')
            spell = spell_checker.check(i)
            checked = spell.checked
            if ' ' in checked:
                hospital_list.append(hos)
                test.append(checked)
    except:
        pass

reviews = pd.DataFrame({'hospital':hospital_list, 'reviews':test})
reviews.head()

  4%|▎         | 1080/29363 [07:28<2:50:10,  2.77it/s]

In [ ]:
words = []

for i in tqdm(data['cleaned_reviews'].tolist()):
    try:
        pos = okt.pos(i)
        # print(pos)
        for p in pos:
            if 'Noun' in p:
                words_ = clean_text(str(p))
                words.append(words_)
            elif 'Verb' in p:
                words_ = clean_text(str(p))
                words_ = okt.morphs(words_,stem=True)
                words_ = clean_text(str(words_))
                words.append(words_)
            # elif 'Adjective' in p:
        #     words_ = clean_text(str(p))
        #     words.append(words_)
    except:
        pass
        

words_df = pd.DataFrame({'words':words})

In [ ]:
cleaned_words=[]

for i in tqdm(words_df['words'].tolist()):
    try:
        if len(i)<=1:
            pass
        else:
            cleaned_words.append(i)
    except:
        pass
    
words_df = pd.DataFrame({'words':cleaned_words})
words_value= pd.DataFrame(words_df['words'].value_counts())


,words
하다,189
병원,92
해주다,81
선생님,70
피부,67
...,...
알아보다,1
운동,1
식이,1
인스턴트,1


In [ ]:
words_df.to_excel('[0512]words_df.xlsx')
words_value.to_excel('[0512]words_value.xlsx')
reviews.to_excel('[0512]reviews.xlsx')